# Dates
*Arthur Ryman, last updated 2025-04-01*

## Introduction

The purpose of this notebook is to develop Python code that implements date functions useful for the 
[Guideline](https://www.fct-cf.gc.ca/en/pages/representing-yourself/deadlines-calculator/guideline)
for calculating Federal Court due dates.

## Federal Court Rules

The following provisions of the 
[Federal Courts Rules](https://laws-lois.justice.gc.ca/eng/regulations/sor-98-106/FullText.html)
will assist you in computing the time within which to file and serve your documents.

* Rule 6(1) incorporates the provisions of sections 26 to 30 of the Interpretation Act for computing time.
* Rule 6(2) governs the computation of periods of less than 7 days.
* Rule 6(3) addresses the exclusion of the seasonal recess in the computation of time provided by the Rules.
* Rules 7 and 8 govern extensions by consent and the Court’s discretion to extend or abridge time periods.
* Holiday means a Saturday, Sunday or any other day defined as a holiday in subsection 35(1) of the Interpretation Act.
* Summer recess means the months of July and August in each year.
* Seasonal recess means the period beginning on December 21 in a year and ending on January 7 in the following year.

Clearly, we need some date and calendar functions.
We need to be able compute the days before or after an event and determine if they are holidays.
Fortunately, Python provides suitable functions.

The Python `datetime` module provides the `datetime.date` object which represents dates and can convert dates to
and from strings. We'll write dates as strings in the ISO `YYYY-MM-DD` format.
The following function makes a `datetime.date` object from an ISO date string.

In [1]:
import datetime
from deadlines.dates import parse_date

PI_DAY = '2025-03-14'
pi_date = parse_date(PI_DAY)

pi_date

datetime.date(2025, 3, 14)

Given a `datetime.date` object, we can convert it back to an ISO string.

In [2]:
from deadlines.dates import format_date

format_date(pi_date)

'2025-03-14'

Given a date, we'll need to compute the days before it and the days after it.
For example, compute 10 days after a given date.

In [3]:
from deadlines.dates import add_days

add_days(pi_date, 10)

datetime.date(2025, 3, 24)

The next date is one day after the given date.

In [4]:
from deadlines.dates import next_date

next_date(pi_date)

datetime.date(2025, 3, 15)

The previous date is one day before the given date.

In [5]:
from deadlines.dates import previous_date

previous_date(pi_date)

datetime.date(2025, 3, 13)

We'll need to determine which day of the week a given day falls on.
This capability will be used to determine if a given date is a weekend or holiday.
Python numbers the days of the weekend from 0 to 6 with 0 being Monday.

In [6]:
from deadlines.dates import weekday_number

pi_weekday_number = weekday_number(pi_date)

pi_weekday_number

4

As of Python 3.12, the `calendar` module provides the `calendar.Day` class to enumerate the weekdays.
However, Google Colab is currently at Python 3.11.11 so I backported it to my package as `Weekday`.
Similarly for `Month`.

In [7]:
from deadlines.enums import Month, Weekday

for day in Weekday:
    print(day.value, day.name)

0 MONDAY
1 TUESDAY
2 WEDNESDAY
3 THURSDAY
4 FRIDAY
5 SATURDAY
6 SUNDAY


We can look up the weekday number in the `calendar.Day` enumeration.

In [8]:
Weekday(pi_weekday_number)

<Weekday.FRIDAY: 4>

We can compare weekday numbers with the enumeration values.

In [9]:
pi_weekday_number == Weekday.FRIDAY

True

In [10]:
pi_weekday_number == Weekday.SATURDAY

False

The Python `calendar` module also defines the names of the weekdays with the usual capitalization.

In [11]:
import calendar

list(calendar.day_name)

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

Given a date, we can look up its weekday name.

In [12]:
from deadlines.dates import weekday_name

weekday_name(pi_date)

'Friday'

The `calendar` module can print the calendar for a given year.

In [13]:
!python -m calendar 2025

                                  2025

      January                   February                   March
Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su
       1  2  3  4  5                      1  2                      1  2
 6  7  8  9 10 11 12       3  4  5  6  7  8  9       3  4  5  6  7  8  9
13 14 15 16 17 18 19      10 11 12 13 14 15 16      10 11 12 13 14 15 16
20 21 22 23 24 25 26      17 18 19 20 21 22 23      17 18 19 20 21 22 23
27 28 29 30 31            24 25 26 27 28            24 25 26 27 28 29 30
                                                    31

       April                      May                       June
Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su
    1  2  3  4  5  6                1  2  3  4                         1
 7  8  9 10 11 12 13       5  6  7  8  9 10 11       2  3  4  5  6  7  8
14 15 16 17 18 19 20      12 13 14 15 16 17 18       9 10 11 12 13 14 15
21 22 23 24 25 26 27      19 20 21 22 23 24 

We can limit the calendar to one month.

In [14]:
!python -m calendar 2025 3

     March 2025
Mo Tu We Th Fr Sa Su
                1  2
 3  4  5  6  7  8  9
10 11 12 13 14 15 16
17 18 19 20 21 22 23
24 25 26 27 28 29 30
31


Note that by default weeks start on Monday. As of Python 3.12, we can change this to Sunday (day number 6) to match the examples given in the Federal Court Guideline document.
However, we need to run on Python 3.11.11 since that is what Google Colab currently supports.

## Deadlines - General

> When a document is to be filed or served within a defined number of days before or after a specified event, 
do not include the day of the event when calculating the deadline, but do include all other days, 
including weekends and holidays (please see exceptions). The deadline includes the last day.

At a high level, the problem is to compute a deadline for filing or serving a document.
We'll generically refer to filing or serving a document as an *action*.
The deadline for the action is triggered by some *event*.

The deadline may be either before or after the event.

For example, the event might be a court ruling and the action might be filing an appeal.
In this case, the deadline is triggered by the date of the court ruling and comes after it.
Presumably, an appeal filed after the deadline would be rejected.

On the other hand, the event might be an upcoming trial in which case the parties may have to file evidence in
advance of the event. 
In this case the deadline is before the event.

Our task is to implement a function that computes the deadline date given the following input data:
* `event_date`: the date of the event
* `deadline_is_after_event`: a boolean flag that is `True` if the deadline is after the event and `False` if the deadline is before the event
* `max_days`: the maximum permissible number of days that can elapse between the event and the required action
* `action_type`: the detailed rules for computing the deadline depend on several factors related to the nature of the action.
We'll have to identity the set of distinct action types and implement the policies associated with each.
We can start with *general* actions.

Next, we consider the examples provided in the Guideline document.

### General Example 1 - 10 Days

To calculate a 10-day deadline after a specific event, 
for example the filing of a document, do not include the first day of the period but do include the 10th day.  
In this example, the period starts on May 31st and the 10th day is a Sunday, so the document is due on the next business day.

<figure style="text-align: center;">
    <img src="images/guideline/general-example-1.png">
    <figcaption>Example 1 - 10 Days</figcaption>
</figure>

The months of May (31 days) and June (30 days) are represented side by side in a calendar view. Thursday May 31st and Monday June 11th are highlighted in yellow to show the 10-day period taking into account the weekend.

#### What year are the example calendars taken from?

We'll use the examples given the Guideline document as test cases for our code.
However, the calendars do not indicate the year.
We need to determine the year so that we can exactly match the examples.

The calendar in Example 1 shows May 1 on a Tuesday.
Find the most recent year as of 2024 in which May 1 was a Tuesday. 

In [15]:
from deadlines.dates import find_year

find_year(Month.MAY, 1, Weekday.TUESDAY, 2024)

2018

Verify this result by printing the calendar for May 2018.

In [16]:
!python -m calendar 2018 5

      May 2018
Mo Tu We Th Fr Sa Su
    1  2  3  4  5  6
 7  8  9 10 11 12 13
14 15 16 17 18 19 20
21 22 23 24 25 26 27
28 29 30 31


The calendar for May 2018 matches the one shown in Example 1.

The event date is 2018-05-31 and the maximum number of days is 10.

In [17]:
event_date = datetime.date(2018, Month.MAY, 31)
deadline_is_after_event = True
max_days = 10
action_type = None

event_date

datetime.date(2018, 5, 31)

### Holidays/Weekends Example 2 - 4 Days

The calendar for this example is not contiguous with the previous examples.

<figure style="text-align: center;">
    <img src="images/guideline/holidays-weekends-example-2.png">
    <figcaption>Holidays/Weekends Example 2 - 4 Days</figcaption>
</figure>

What year is this?

In [18]:
find_year(Month.APRIL, 1, Weekday.SUNDAY, 2024)

2018

In [19]:
!python -m calendar 2018 4

     April 2018
Mo Tu We Th Fr Sa Su
                   1
 2  3  4  5  6  7  8
 9 10 11 12 13 14 15
16 17 18 19 20 21 22
23 24 25 26 27 28 29
30


Although the dates match for 2018, the holidays don't. 
A web search reveals that the most recent year in which Good Friday fell on April 6 was 2012.

Calculate Good Friday for 2012.

In [20]:
from deadlines.canadian_holidays import *

calc_good_friday(2012)

datetime.date(2012, 4, 6)

Print the calendar for 2012 and compare it to all the examples.

In [21]:
!python -m calendar 2012

                                  2012

      January                   February                   March
Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su
                   1             1  2  3  4  5                1  2  3  4
 2  3  4  5  6  7  8       6  7  8  9 10 11 12       5  6  7  8  9 10 11
 9 10 11 12 13 14 15      13 14 15 16 17 18 19      12 13 14 15 16 17 18
16 17 18 19 20 21 22      20 21 22 23 24 25 26      19 20 21 22 23 24 25
23 24 25 26 27 28 29      27 28 29                  26 27 28 29 30 31
30 31

       April                      May                       June
Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su
                   1          1  2  3  4  5  6                   1  2  3
 2  3  4  5  6  7  8       7  8  9 10 11 12 13       4  5  6  7  8  9 10
 9 10 11 12 13 14 15      14 15 16 17 18 19 20      11 12 13 14 15 16 17
16 17 18 19 20 21 22      21 22 23 24 25 26 27      18 19 20 21 22 23 24
23 24 25 26 27 28 29   

The calendar for 2012 matches all the General examples since May 1 is a Tuesday, both Holiday/Weekend examples
since April 1 is a Sunday, the Summer Recess example since June 1 is a Friday, 
and the first 2 Seasonal Recess examples since December 1 is a Saturday.
The third Seasonal Recess example has December 1 on Wednesday.

### Seasonal Recess Example 1 - 30 Days

The calendar for this example is not contiguous with the previous examples.

<figure style="text-align: center;">
    <img src="images/guideline/seasonal-recess-example-1.png">
    <figcaption>Seasonal Recess Example 1 - 30 Days</figcaption>
</figure>

What year is this?

In [22]:
find_year(Month.DECEMBER, 1, Weekday.SATURDAY, 2024)

2018

This example also matches 2018. 
Seasonal Recess Example 2 has the same calendar.

Print the calendar to confirm this.

In [23]:
!python -m calendar 2018 12

   December 2018
Mo Tu We Th Fr Sa Su
                1  2
 3  4  5  6  7  8  9
10 11 12 13 14 15 16
17 18 19 20 21 22 23
24 25 26 27 28 29 30
31


### Seasonal Recess Example 3 - 4 Days

The calendar for this example is not contiguous with the previous examples.

<figure style="text-align: center;">
    <img src="images/guideline/seasonal-recess-example-3.png">
    <figcaption>Seasonal Recess Example 3 - 4 Days</figcaption>
</figure>

What year is this?

In [24]:
find_year(Month.DECEMBER, 1, Weekday.WEDNESDAY, 2024)

2021

Print the calendar to confirm this.

In [25]:
!python -m calendar 2021 12

   December 2021
Mo Tu We Th Fr Sa Su
       1  2  3  4  5
 6  7  8  9 10 11 12
13 14 15 16 17 18 19
20 21 22 23 24 25 26
27 28 29 30 31


## Summary

All examples except the last one match 2012. The last example matches 2021. 
This is plausible because the web page has a last modification date of 2023-09-22.
Whoever added the last example didn't update the calendar figure consistentlt because its caption is "Example 2" but it
should be "Example 3".

## Next Steps

At this point we have enough generic date functions and have identified the correct years for each example.
Next, implement the detailed Federal Rules and test the code against all these examples.